# Amplitude Estimation class

In order to give an easy access to the user to the different **AE** algorithm class a selection class called *AE* was implemented into the module *ae_class* within the package *AE* of library *QQuantLib* (**QQuantLib/AE/ae_class.py**).

Present notebook review in a quick way the working of this class

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --depth 

For more information about loading data into the quantum circuit see the notebook *01_DataLoading_Module_Use*.

## 2. AE class

The **AE** class allows the user to solve an **Amplitude Estimation** problem by providing and oracle and the type of **AE** algorithm user want to use.

### 2.1 The Amplitude Estimation Problem

In a general way an amplitude estimation problem is the following: given an oracle:

$$\mathcal{O}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.  

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $a$.  We can define an associated angle to $\sqrt{a}$ as $\sin^2{\theta} = a$, and the problem is thus rewritten as:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$

The foundations of any amplitude estimation algorithm is the grover operator $\mathcal{Q}$, built onto the oracle $\mathcal{O}$, that has the following effect over our state $|\Psi\rangle$:

$$\mathcal{G}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$

for more information about the grover operator and the amplitude amplification algorithm check the notebook **02_AmplitudeAmplification_Operators.ipynb**.


### 2.2 Creating object from AE class

We have implemented a python class called **AE** into the **QQuantLib/AE/ae_class** module that allows us to use the all the AE algorithms implemented in the **QQuantLib.AE** package in a transparent way for the user. 

When class is created following mandatory inputs MUST be provided:

We have some mandatory inputs:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binnary representation as a python list
3. index: list of the qbits affected by the Grover operator.
4. ae_type: str. String for selecting the algorithm to use. Valid values
    * **MLAE**:  for using Maximum Likelihood algorithm (see Notebook 03_Maximum_Likelihood_Amplitude_Estimation_Class.ipynb)
    * **CQPEAE**: for using classical Phase Estimation with Quantum Fourier Transformation (**QFT**) for **AE** (see Notebook 04_Classical_Phase_Estimation_Class.ipynb)
    * **IQPEAE**: for using classical Phase Estimation with iterative Quantum Fourier Transformation (**QFT**) for **AE** (see Notebook 05_Iterative_Quantum_Phase_Estimation_Class.ipynb)
    * **IQAE**: for using Iterative Quantum Amplitude Estimation (see Notebook 06_Iterative_Quantum_Amplitude_Estimation_class.ipynb).
    * **RQAE**: for using Real Quantum Amplitude Estimation (see Notebook 07_Real_Quantum_Amplitude_Estimation_class.ipynb)
    
If *Oracle*, *target* or *index* is not provide a ValueError is raised.
Additionally, other optional inputs can be provided as a python dictionary for configuring the selected algorithms (see the Notebooks before for the each **AE** algorithm parameters. 

In [ ]:
from QQuantLib.AE.ae_class import AE

As example we are going to use the example of section 1 where the estimation problem where the estimation problem will be:

$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

The target state, in this case is $|1\rangle$. It's binary representation is $001$. This has to be passed to the target variable as a list. Moreover we have to provide the list of qubits where we are acting, in this case is just $[0,1,2]$, the whole register.

In [ ]:
target = [0,0,1]
index = [0,1,2]

In [ ]:
from QQuantLib.utils.utils import bitfield_to_int
desired_result = probability[bitfield_to_int(target)]
print("desired_result: {}".format(desired_result))

The python dictionary can be a very general one where all the parameters for configuring any algorithm can be provided. The class select for each algorithm the parameters it needs. If there is a parameter not provided for a method the default parameter of the method will be used!!

In [ ]:
m_k = [i for i in range(12)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multicontrolate decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    #'q': 1.2
}

In [ ]:
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

The *ae_type* parameter can not be provided initially and can be set in posterior way.

### 2.3 The create_ae_solver method

The *create_ae_solver* method allows the user select the **AE** class. When this method is called following attributes are created:

* *solver_dict*: python dictionary with the parameters for configuring the selected **AE** class (for selecting the class use the *ae_type* atribute). Only the parameters needed by the class and provided in the input dictionary will be included. If the selected **ae** class need any parameter that is not included in  the input python dictionary the default value of the class will be used.
* *solver_ae*: this is an object created from the different **AE** classes of the **QQuantLib.AE** package (for selecting the class use the *ae_type* atribute). For configuring the methods the *solver_dict* will be used.


If *ae_type* is None the *create_ae_solver* method will raise an Error.



In [ ]:
#ae_type is None so this cell raises an error
ae_object.create_ae_solver()

In [ ]:
#this is None
ae_object.solver_dict

In [ ]:
#Now we select the MLAE method
ae_object.ae_type = 'MLAE'
ae_object.create_ae_solver()

In [ ]:
#Configuration for the AE class
ae_object.solver_dict

In [ ]:
#For configuring the parameters of the method the keys from the solver_dict will be used
print('From the method: {}'.format(ae_object.solver_ae.schedule))

print('From the solver_dict attribute: {}'.format(ae_object.solver_dict['schedule']))

We can change the **AE** algorithm to use

In [ ]:
ae_object.ae_type = 'IQPEAE'
ae_object.create_ae_solver()

In [ ]:
ae_object.solver_ae

In [ ]:
#Configuration for the AE class
ae_object.solver_dict

In [ ]:
ae_object.ae_type = 'RQAE'
ae_object.create_ae_solver()
#In this case the q parameter is not provided in the input dictionary so it doesn't appears in the solver_dict
ae_object.solver_dict

In [ ]:
#The default value for the ae class vill be used instead:
ae_object.solver_ae.ratio

### 2.4 Executing the Algorithm

The *solver_ae* atrribute is an object of the desired **AE** class so we can use directly the **run** method for executing the algorithm

In [ ]:
m_k = [i for i in range(12)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multicontrolate decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2
}

In [ ]:
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

In [ ]:
ae_object.ae_type = 'MLAE'
ae_object.create_ae_solver()

In [ ]:
ae_object.solver_ae.run()

Additionally all the atributes of the class are avialbale:

In [ ]:
ae_object.solver_ae.h_k

In [ ]:
ae_object.solver_ae.oracle_calls

In [ ]:
ae_object.solver_ae.schedule_pdf

We can change now the **AE** algorithm

In [ ]:
ae_object.ae_type = 'IQAE'
ae_object.create_ae_solver()
ae_object.solver_ae.run()

Again the particular attributes of the solver can be accesed in an easy way

In [ ]:
[ae_object.solver_ae.ae, ae_object.solver_ae.ae_l, ae_object.solver_ae.ae_u]

In [ ]:
ae_object.solver_ae.oracle_calls

In [ ]:
ae_object.solver_ae.schedule_pdf

## 3. AE complete execution

The **AE** class have a *run* method for executing directly the solver with a desired input. In this case the method select the **ae** class and execute its run method. The results are stored in the atribute *ae_pdf* which is a pandas DataFrame with the direct result of the ae. This DataFrame have 3 columns:
* ae: is the desired $\sqrt{a}$
* ae_l: is the lower bound for the $\sqrt{a}$ (only for **IQAE** and **RQAE**)
* ae_u: is the upper boudn for the $\sqrt{a}$ (only for **IQAE** and **RQAE**) 

Additionally the *run* method populates the *oracle_calls* and the *max_oracle_depth* attributes:

* *oracle_calls*: number of total oracle calls for a complete execution of the algorithm
* *max_oracle_depth*: maximum number of applications of the oracle for the complete execution of the algorithm.


In [ ]:
m_k = [i for i in range(12)]
ae_dict = {
    #QPU
    'qpu': linalg_qpu,
    #Multicontrolate decomposition
    'mcz_qlm': False, 
    
    #shots
    'shots': 100,
    
    #MLAE
    'schedule': [
        m_k,
        [100]*len(m_k)
    ],
    'delta' : 1.0e-6,
    'ns' : 10000,
    
    #CQPEAE
    'auxiliar_qbits_number': 10,
    #IQPEAE
    'cbits_number': 6,
    #IQAE & RQAQE
    'epsilon': 0.0001,
    #IQAE
    'alpha': 0.05,
    #RQAE
    'gamma': 0.05,
    'q': 1.2
}
ae_object = AE(
    oracle=oracle,
    target=target,
    index=index,
    **ae_dict)

### 3.1 MLAE

In [ ]:
ae_object.ae_type = 'MLAE'

In [ ]:
ae_object.run()

In [ ]:
mlae_result = ae_object.ae_pdf
mlae_result

In [ ]:
print("Absolute Error for MLAE: {}".format(np.abs(mlae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.solver_ae.schedule_pdf

### 3.2 CQPEAE

In [ ]:
ae_object.ae_type = 'CQPEAE'
ae_object.run()
cqpeae_result = ae_object.ae_pdf

In [ ]:
cqpeae_result

In [ ]:
print("Absolute Error for CQPEAE: {}".format(np.abs(cqpeae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.solver_ae.schedule_pdf

### 3.3 IQPEAE

In [ ]:
ae_object.ae_type = 'IQPEAE'
ae_object.run()
iqpeae_result = ae_object.ae_pdf

In [ ]:
iqpeae_result

In [ ]:
print("Absolute Error for IQPEAE: {}".format(np.abs(iqpeae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

### 3.4 IQAE

In [ ]:
ae_object.ae_type = 'IQAE'
ae_object.run()
iqae_result = ae_object.ae_pdf

In [ ]:
iqae_result

In [ ]:
print("Absolute Error for IQAE: {}".format(np.abs(iqae_result['ae'][0] - desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.solver_ae.schedule_pdf

### 3.5 RQAE

**BE AWARE** The RQAE algorithm provides directly the **Amplitude** of the state so we need to compare vs the square root!!

$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & a|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a^2}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

In [ ]:
ae_object.ae_type = 'RQAE'
ae_object.run()
rqae_result = ae_object.ae_pdf

In [ ]:
rqae_result**2

In [ ]:
#in this case the desired result is 

In [ ]:
print("Absolute Error for RQAE: {}".format(np.abs(rqae_result['ae'][0]**2 -desired_result)))

In [ ]:
ae_object.oracle_calls

In [ ]:
ae_object.max_oracle_depth

In [ ]:
ae_object.solver_ae.schedule_pdf